# Dental X-Ray Denoising with Autoencoder

Matthew Lucas  
Unit 4 Incremental Capstone  
Class 2509 TA

**Objective:** Build an autoencoder to remove noise from dental X-ray images. The model will learn to reconstruct clean images from noisy versions.


## Imports


In [ ]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Reproducibility
# TODO: Set SEED to 42
SEED = ___
np.random.seed(SEED)
tf.random.set_seed(SEED)

print("TensorFlow version:", tf.__version__)


## Load and Preprocess Dataset


In [ ]:
# Load the dataset
# TODO: Use np.load() to load 'DENTAL_1.NPZ'
data = np.___('DENTAL_1.NPZ')

# Extract arrays
# TODO: Extract 'x_train', 'x_test', 'y_train', 'y_test' from the loaded data
X_train_clean = data[___]
X_test_clean = data[___]
y_train = data[___]
y_test = data[___]

print("Dataset shapes:")
print(f"Training images: {X_train_clean.shape}")
print(f"Test images: {X_test_clean.shape}")
print(f"Training labels: {y_train.shape}")
print(f"Test labels: {y_test.shape}")

# Check original pixel value range
print(f"\nOriginal pixel value range:")
print(f"Training: min={X_train_clean.min():.3f}, max={X_train_clean.max():.3f}")
print(f"Test: min={X_test_clean.min():.3f}, max={X_test_clean.max():.3f}")

# Convert to float32 (data is already normalized to [0, 1] range)
# Do NOT divide by 255 - the data is already normalized!
# TODO: Convert X_train_clean and X_test_clean to float32
X_train_clean = X_train_clean.astype(___)
X_test_clean = X_test_clean.astype(___)

print(f"\nAfter conversion to float32:")
print(f"Training: min={X_train_clean.min():.3f}, max={X_train_clean.max():.3f}")
print(f"Test: min={X_test_clean.min():.3f}, max={X_test_clean.max():.3f}")


## Add Gaussian Noise


In [ ]:
# Add Gaussian noise to create noisy versions of the images
# This simulates real-world noise in X-ray images
# TODO: Set NOISE_FACTOR to 0.3 (controls the amount of noise)
NOISE_FACTOR = ___

def add_noise(images, noise_factor=NOISE_FACTOR):
    """
    Add Gaussian noise to images.
    
    Args:
        images: Clean images (normalized to [0, 1])
        noise_factor: Standard deviation of the noise (relative to image range)
    
    Returns:
        Noisy images
    """
    # Generate noise with same shape as images
    # TODO: Use np.random.normal() to generate noise
    # Parameters: loc=0.0, scale=noise_factor, size=images.shape
    noise = np.random.normal(loc=___, scale=___, size=___)
    
    # Add noise and clip to valid range [0, 1]
    # TODO: Add noise to images and clip to [0.0, 1.0] using np.clip()
    noisy_images = images + ___
    noisy_images = np.clip(___, ___, ___)
    
    return noisy_images

# Create noisy versions of training and test images
# TODO: Call add_noise() for both training and test images
X_train_noisy = add_noise(___, noise_factor=___)
X_test_noisy = add_noise(___, noise_factor=___)

print(f"Created noisy images with noise factor: {NOISE_FACTOR}")
print(f"Training noisy images shape: {X_train_noisy.shape}")
print(f"Test noisy images shape: {X_test_noisy.shape}")

# Display sample images to see the noise
fig, axes = plt.subplots(2, 4, figsize=(12, 6))
for i in range(4):
    # Clean images (top row)
    axes[0, i].imshow(X_train_clean[i])
    axes[0, i].set_title('Clean')
    axes[0, i].axis('off')
    
    # Noisy images (bottom row)
    axes[1, i].imshow(X_train_noisy[i])
    axes[1, i].set_title('Noisy')
    axes[1, i].axis('off')

plt.suptitle('Sample Images: Clean (top) vs Noisy (bottom)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()


## Build Autoencoder Model


In [ ]:
# Build autoencoder for image denoising
# Input shape: (256, 256, 3) - RGB images

# Encoder: Compresses the image to a lower-dimensional representation
# TODO: Build the encoder with Conv2D and MaxPooling2D layers
# Layer 1: Conv2D with 32 filters, (3,3) kernel, 'relu' activation, 'same' padding
# Layer 2: MaxPooling2D with (2,2) pool size, 'same' padding
# Layer 3: Conv2D with 64 filters, (3,3) kernel, 'relu' activation, 'same' padding
# Layer 4: MaxPooling2D with (2,2) pool size, 'same' padding
# Layer 5: Conv2D with 128 filters, (3,3) kernel, 'relu' activation, 'same' padding
# Layer 6: MaxPooling2D with (2,2) pool size, 'same' padding
encoder = models.Sequential([
    layers.Input(shape=(256, 256, 3)),
    layers.Conv2D(___, (___, ___), activation=___, padding=___),
    layers.MaxPooling2D((___, ___), padding=___),  # 128x128
    layers.Conv2D(___, (___, ___), activation=___, padding=___),
    layers.MaxPooling2D((___, ___), padding=___),  # 64x64
    layers.Conv2D(___, (___, ___), activation=___, padding=___),
    layers.MaxPooling2D((___, ___), padding=___),  # 32x32
])

# Decoder: Reconstructs the image from the compressed representation
# TODO: Build the decoder with Conv2DTranspose and Conv2D layers
# Layer 1: Conv2DTranspose with 128 filters, (3,3) kernel, strides=2, 'relu' activation, 'same' padding
# Layer 2: Conv2DTranspose with 64 filters, (3,3) kernel, strides=2, 'relu' activation, 'same' padding
# Layer 3: Conv2DTranspose with 32 filters, (3,3) kernel, strides=2, 'relu' activation, 'same' padding
# Layer 4: Conv2D with 3 filters, (3,3) kernel, 'sigmoid' activation, 'same' padding (output layer)
decoder = models.Sequential([
    layers.Conv2DTranspose(___, (___, ___), strides=___, activation=___, padding=___),  # 64x64
    layers.Conv2DTranspose(___, (___, ___), strides=___, activation=___, padding=___),   # 128x128
    layers.Conv2DTranspose(___, (___, ___), strides=___, activation=___, padding=___),  # 256x256
    layers.Conv2D(___, (___, ___), activation=___, padding=___)  # Output: 256x256x3
])

# Combine encoder and decoder
# TODO: Create the autoencoder by combining encoder and decoder
autoencoder = models.Sequential([___, ___])

# Compile the model
# TODO: Compile with optimizer='adam', loss='mse', metrics=['mae']
autoencoder.compile(
    optimizer=___,
    loss=___,  # Mean Squared Error for reconstruction
    metrics=[___]  # Mean Absolute Error
)

autoencoder.summary()


## Train the Autoencoder


In [ ]:
# Set up callbacks
# TODO: Create EarlyStopping callback with:
# - monitor='val_loss'
# - patience=10
# - restore_best_weights=True
# - verbose=1
callbacks = [
    EarlyStopping(
        monitor=___,
        patience=___,
        restore_best_weights=___,
        verbose=___
    ),
    # TODO: Create ReduceLROnPlateau callback with:
    # - monitor='val_loss'
    # - factor=0.5
    # - patience=5
    # - min_lr=1e-7
    # - verbose=1
    ReduceLROnPlateau(
        monitor=___,
        factor=___,
        patience=___,
        min_lr=___,
        verbose=___
    )
]

# Train the autoencoder
# Input: noisy images, Target: clean images
# The model learns to reconstruct clean images from noisy ones
# TODO: Fill in model.fit() parameters:
# - X_train_noisy (input: noisy images)
# - X_train_clean (target: clean images)
# - validation_data=(X_test_noisy, X_test_clean)
# - epochs=50
# - batch_size=8
# - callbacks=callbacks
# - verbose=1
history = autoencoder.fit(
    ___,  # Input: noisy images
    ___,  # Target: clean images
    validation_data=(___, ___),
    epochs=___,
    batch_size=___,
    callbacks=___,
    verbose=___
)


## Evaluate and Visualize Results


In [ ]:
# Evaluate on test set
# TODO: Use autoencoder.evaluate() on X_test_noisy and X_test_clean
# Set verbose=0
test_loss, test_mae = autoencoder.___(___, ___, verbose=___)
print(f"Test Loss (MSE): {test_loss:.6f}")
print(f"Test MAE: {test_mae:.6f}")

# Generate predictions (denoised images)
# TODO: Use autoencoder.predict() on X_test_noisy
# Set verbose=0
denoised_images = autoencoder.___(___, verbose=___)

print(f"\nDenoised images shape: {denoised_images.shape}")
print(f"Denoised pixel range: [{denoised_images.min():.3f}, {denoised_images.max():.3f}]")


In [ ]:
# Visualize results: Compare noisy, denoised, and clean images
n_samples = 8
fig, axes = plt.subplots(3, n_samples, figsize=(16, 6))

for i in range(n_samples):
    # Row 1: Noisy images (input)
    axes[0, i].imshow(X_test_noisy[i])
    axes[0, i].set_title('Noisy (Input)')
    axes[0, i].axis('off')
    
    # Row 2: Denoised images (model output)
    axes[1, i].imshow(denoised_images[i])
    axes[1, i].set_title('Denoised (Output)')
    axes[1, i].axis('off')
    
    # Row 3: Clean images (ground truth)
    axes[2, i].imshow(X_test_clean[i])
    axes[2, i].set_title('Clean (Ground Truth)')
    axes[2, i].axis('off')

plt.suptitle('Denoising Results: Noisy → Denoised → Clean', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()


In [ ]:
# Calculate reconstruction error for each test image
# TODO: Calculate MSE per image using np.mean() on (X_test_clean - denoised_images) ** 2
# Use axis=(1, 2, 3) to average across height, width, and channels
mse_per_image = np.mean((___ - ___) ** 2, axis=(___, ___, ___))

# TODO: Calculate MAE per image using np.mean() on np.abs(X_test_clean - denoised_images)
# Use axis=(1, 2, 3) to average across height, width, and channels
mae_per_image = np.mean(np.abs(___ - ___), axis=(___, ___, ___))

print("Reconstruction Error Statistics:")
print(f"Mean MSE per image: {np.mean(mse_per_image):.6f}")
print(f"Mean MAE per image: {np.mean(mae_per_image):.6f}")
print(f"\nBest reconstruction (lowest MSE): Image {np.argmin(mse_per_image)}")
print(f"Worst reconstruction (highest MSE): Image {np.argmax(mse_per_image)}")

# Visualize best and worst reconstructions
fig, axes = plt.subplots(2, 3, figsize=(12, 8))

# Best reconstruction
# TODO: Find the index of the best reconstruction using np.argmin() on mse_per_image
best_idx = np.___(___)
axes[0, 0].imshow(X_test_noisy[best_idx])
axes[0, 0].set_title(f'Noisy (MSE: {mse_per_image[best_idx]:.6f})')
axes[0, 0].axis('off')

axes[0, 1].imshow(denoised_images[best_idx])
axes[0, 1].set_title('Denoised')
axes[0, 1].axis('off')

axes[0, 2].imshow(X_test_clean[best_idx])
axes[0, 2].set_title('Clean (Ground Truth)')
axes[0, 2].axis('off')

# Worst reconstruction
# TODO: Find the index of the worst reconstruction using np.argmax() on mse_per_image
worst_idx = np.___(___)
axes[1, 0].imshow(X_test_noisy[worst_idx])
axes[1, 0].set_title(f'Noisy (MSE: {mse_per_image[worst_idx]:.6f})')
axes[1, 0].axis('off')

axes[1, 1].imshow(denoised_images[worst_idx])
axes[1, 1].set_title('Denoised')
axes[1, 1].axis('off')

axes[1, 2].imshow(X_test_clean[worst_idx])
axes[1, 2].set_title('Clean (Ground Truth)')
axes[1, 2].axis('off')

plt.suptitle('Best (top) and Worst (bottom) Reconstructions', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()


## Plot Training History


In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Loss plot
# TODO: Plot history.history['loss'] and history.history['val_loss']
axes[0].plot(history.history[___], label='Training Loss')
axes[0].plot(history.history[___], label='Validation Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss (MSE)')
axes[0].set_title('Model Loss')
axes[0].legend()
axes[0].grid(alpha=0.3)

# MAE plot
# TODO: Plot history.history['mae'] and history.history['val_mae']
axes[1].plot(history.history[___], label='Training MAE')
axes[1].plot(history.history[___], label='Validation MAE')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Mean Absolute Error')
axes[1].set_title('Model MAE')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()


## Questions to Consider

Reflect on what you've learned and explore further:

### Understanding the Model

1. **How does an autoencoder work?** Explain the difference between the encoder and decoder components. What happens to the image dimensions as it passes through the encoder? How does the decoder reconstruct the image?

2. **Why use MSE (Mean Squared Error) as the loss function?** What does MSE measure, and why is it appropriate for image reconstruction tasks? How would the results differ if you used a different loss function?

3. **What role does the bottleneck play?** The encoder compresses the image from 256×256×3 down to 32×32×128. What information is preserved in this compressed representation? What information might be lost?

### Model Performance

4. **Analyze the reconstruction quality:** Compare the best and worst reconstructions. What characteristics make some images easier to denoise than others? Are there patterns in the images that the model struggles with?

5. **Noise factor impact:** How would changing the `NOISE_FACTOR` affect the model's performance? Try different values (e.g., 0.1, 0.5, 0.7) and observe how the denoising quality changes. What happens when the noise is too strong?

6. **Training observations:** Look at the training history plots. Did the model converge? Were there signs of overfitting or underfitting? How did the validation loss compare to training loss?

### Experimentation Ideas

7. **Architecture modifications:** Try modifying the autoencoder architecture:
   - Add more layers to the encoder/decoder
   - Change the number of filters in each layer
   - Add dropout layers to prevent overfitting
   - Experiment with different activation functions

8. **Different noise types:** Instead of Gaussian noise, try:
   - Salt-and-pepper noise
   - Poisson noise (common in medical imaging)
   - Motion blur
   - How does the model perform with different noise types?

9. **Loss function alternatives:** Experiment with different loss functions:
   - Perceptual loss (using a pre-trained network)
   - SSIM (Structural Similarity Index)
   - Combination of MSE and MAE
   - How do these affect reconstruction quality?

10. **Real-world applications:** Where else could autoencoders be useful?
    - Image compression
    - Anomaly detection
    - Feature extraction
    - Data augmentation
    - What other medical imaging tasks could benefit from denoising?

### Further Exploration

11. **Compare with other methods:** Research traditional image denoising techniques (e.g., Gaussian blur, median filtering, bilateral filtering). How do they compare to the autoencoder approach? What are the advantages and disadvantages of each?

12. **Variational Autoencoders (VAEs):** Research VAEs and how they differ from standard autoencoders. What additional capabilities do they provide? When would you choose a VAE over a standard autoencoder?

13. **Transfer learning:** Could you use a pre-trained encoder (e.g., from ImageNet) and only train the decoder? How would this affect training time and performance?

### Critical Thinking

14. **Limitations:** What are the limitations of this autoencoder approach? When might it fail? Are there scenarios where denoising might remove important details from medical images?

15. **Ethical considerations:** In medical imaging, image quality can affect diagnosis. What are the ethical implications of using AI to denoise medical images? Should denoised images be clearly labeled as processed?
